In [1]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import os
import re
import tempfile

# ─────────────────────────────────────
# ✅ 1. Chrome 설정
# ─────────────────────────────────────
temp_dir = tempfile.mkdtemp()

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(f'--user-data-dir={temp_dir}')

driver = webdriver.Chrome(options=options)

# ─────────────────────────────────────
# ✅ 2. 로그인
# ─────────────────────────────────────
driver.get("https://www.groove4.co.kr/member/login.asp")

USER_ID = "summit2025"
USER_PW = "sumsum12!@"

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "login_id"))
).send_keys(USER_ID)

driver.find_element(By.NAME, "login_pw").send_keys(USER_PW)

WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//input[@alt="로그인"]'))
).click()

time.sleep(3)

# ─────────────────────────────────────
# ✅ 3. 예약 페이지 이동
# ─────────────────────────────────────
driver.get("https://www.groove4.co.kr/reservation/reservation.asp?shop=30&gubun=sadang")
time.sleep(3)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 8.3 MB/s eta 0:00:00


In [2]:
# ─────────────────────────────────────
# ✅ 4. 날짜별 클릭 + 데이터 수집 반복
# ─────────────────────────────────────
all_reserved_data = []

# 30일치 자료 크롤링
for i in range(1,30):
    # 새로 DOM을 불러와야 하므로 매번 다시 불러옴
    date_links = driver.find_elements(By.XPATH, "//a[contains(@onclick, 'setReserveDate')]")
    tag = date_links[i]
    onclick_attr = tag.get_attribute("onclick")
    date_str = onclick_attr.split("'")[1]  # '2025-04-02' 같은 형태

    try:
        tag.click()
    except Exception as e:
        print(f"❌ 날짜 클릭 실패: {date_str} ({e})")
        continue

    print(f"📅 {date_str} 로딩 중...")
    time.sleep(2)  # 날짜 바뀌는 시간 대기

    # 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")
    reserve_elements = soup.select(".reserve_time_off")

    for el in reserve_elements:
        el_id = el.get("id", "")
        match = re.match(r"reserve_time_(1[4-7])_([7-9]|1[0-9]|2[0-3])", el_id)
        if match:
            room = int(match.group(1))
            time_num = int(match.group(2))
            formatted_time = f"{time_num:02d}:00 ~ {time_num+1:02d}:00"
            all_reserved_data.append({
                "날짜": date_str,
                "방번호": room,
                "시간": formatted_time
            })

print(f"✅ 총 {len(all_reserved_data)}건 수집 완료")



📅 2025-04-03 로딩 중...
📅 2025-04-04 로딩 중...
📅 2025-04-05 로딩 중...
📅 2025-04-06 로딩 중...
📅 2025-04-07 로딩 중...
📅 2025-04-08 로딩 중...
📅 2025-04-09 로딩 중...
📅 2025-04-10 로딩 중...
📅 2025-04-11 로딩 중...
📅 2025-04-12 로딩 중...
📅 2025-04-13 로딩 중...
📅 2025-04-14 로딩 중...
📅 2025-04-15 로딩 중...
📅 2025-04-16 로딩 중...
📅 2025-04-17 로딩 중...
📅 2025-04-18 로딩 중...
📅 2025-04-19 로딩 중...
📅 2025-04-20 로딩 중...
📅 2025-04-21 로딩 중...
📅 2025-04-22 로딩 중...
📅 2025-04-23 로딩 중...
📅 2025-04-24 로딩 중...
📅 2025-04-25 로딩 중...
📅 2025-04-26 로딩 중...
📅 2025-04-27 로딩 중...
📅 2025-04-28 로딩 중...
📅 2025-04-29 로딩 중...
📅 2025-04-30 로딩 중...
📅 2025-05-01 로딩 중...
✅ 총 1078건 수집 완료


In [3]:
# ─────────────────────────────────────
# ✅ 5. JSON 저장
# ─────────────────────────────────────
output_path = "/content/all_possible_reservation.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_reserved_data, f, ensure_ascii=False, indent=2)

print(f"✅ 예약 정보가 {output_path} 에 저장되었습니다.")

# 브라우저 종료
driver.quit()


✅ 예약 정보가 /content/all_possible_reservation.json 에 저장되었습니다.
